# Generating my dataset

In [19]:
import cv2

def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        
        if faces is ():
            return None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h,x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture(1) # external camera = 1 and 0 for internal
    img_id = 0
    
    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            #ile_name_path = "data/"+"tanaka."+str(img_id)+".jpg"
            file_name_path = "data/"+str(img_id)+'.jpg'
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2 )
            
            cv2.imshow("Cropped_Face", face)
            if cv2.waitKey(1) == 13 or int(img_id) == 1000:    # for 20 for test case
                break

    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed !!!")
generate_dataset()

KeyboardInterrupt: 

# Create Label

In [ ]:
import numpy as np # pip install numpy
import cv2

def my_label(image_name):
    name = image_name.split('.')[-3]
    # depends with the number of people in the dataset
    if name == "Poso":
        return np.array([1,0,0,0])
    elif name == "tanaka":
        return np.array([0,1,0,0])
    elif name == "albright":
        return np.array([0,0,1,0])
    elif name == "sam":
        return np.array([0,0,0,1])

# create data

In [11]:
import os
from random import shuffle
from tqdm import tqdm

def my_data():
    data = []
    for img in tqdm(os.listdir("data")):
        path = os.path.join("data", img)
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (50,50))
        data.append([np.array(img_data), my_label(img)])
    shuffle(data)  #1,2,3,4 to 4,3,1,2
    return data

In [12]:
data = my_data()

  0%|                                                                                         | 0/4000 [00:00<?, ?it/s]


NameError: name 'my_label' is not defined

In [13]:
train = data[:3200] # 80% of the dataset
test = data[3200:] #20% of the dataset
X_train = np.array([i[0] for i in train]).reshape(-1,50,50,1)
print(X_train.shape)
y_train = [i[1] for i in train]
X_test = np.array([i[0] for i in test]).reshape(-1,50,50,1)
print(X_test.shape)
y_test = [i[1] for i in test]

NameError: name 'data' is not defined

# Creating the model

In [14]:
#import warnings
#warnings.filterwarnings('ignore')

import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

tf.reset_default_graph()
convnet = input_data(shape = [50,50,1])
convnet = conv_2d(convnet, 32, 5, activation = 'relu')
# 32 filters and stride = 5 so that the filter will move 5 pixel or unit at a time
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation = 'relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 128, 5, activation = 'relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation = 'relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 32, 5, activation = 'relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation = 'relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 4, activation = 'softmax') # 4 because of 4 authorized people in dataset
convnet = regression(convnet, optimizer = 'adam', learning_rate = 0.001, loss = 'categorical_crossentropy') # stochastic gradiant disseminator
model = tflearn.DNN(convnet, tensorboard_verbose = 3) # from 1 to 3 with 3 having more detail
model.fit(X_train, y_train, n_epoch = 5, validation_set = (X_test, y_test), show_metric = True, run_id = "Auth") # of epoch from 10 to 1000 better accuracy

model.save('my_model.h5')

NameError: name 'X_train' is not defined

# Lets visualize the data and make a prediction

In [15]:
import os
from random import shuffle
from tqdm import tqdm
import numpy as np

def data_for_test():
    Vdata = []
    for img in tqdm(os.listdir("Images for test")):
        path = os.path.join("Images for test", img)
        img_num = img.split('.')[0]
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (50,50))
        Vdata.append([np.array(img_data), img_num])
    shuffle(Vdata)
    return Vdata

In [16]:
Vdata = data_for_test()

100%|█████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 162.29it/s]


In [17]:
import matplotlib.pyplot as plt

test_set = tf.keras.models.load_model("my_model.h5")
fig = plt.figure(figsize = (20,20))
for num, data in enumerate(Vdata[:20]):
    img_data = data[0]
    y = fig.add_subplot(5, 5, num + 1)
    image = img_data
    data = img_data.reshape(50,50,1)
    model_out = test_set.predict([data])[0]
    #[0.8, 0.1, 0.1]
    #[0.2, 0.6, 0.2]
    #[0.1, 0.2, 0.8]
    if np.argmax(model_out) == 0:
        my_label = 'Poso'
    elif np.argmax(model_out) == 1:
        my_label = 'tanaka'
    elif np.argmax(model_out) == 2:
        my_label = 'albright'
    else:
        my_label = 'sam'
            
    y.imshow(image, cmap = 'gray')
    plt.title(my_label)
    
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()

OSError: Unable to open file (unable to open file: name = 'my_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)